In [1]:
import json
import time
from pathlib import Path
from schema_mapping_validator import process_combinations
from fragility_mapping_checker import count_model_ids, check_fragility_mapping

# Define paths to required files

This step defines the necessary input files for the schema mapping validation process and creates an output directory
where all results will be saved. Ensure that the paths to the input files (mapping script, JSON schema, and fragility
CSV) are correctly set before running the notebook.

In [2]:
mapping_script = "mapping_modules/HAZUS_EQ/mapping_IM.py"
mapping_function = "auto_populate"
json_schema_file = "mapping_modules/HAZUS_EQ/input_schema.json"
fragility_csv = "fragility_database/fragility.csv"

output_dir = Path("output_files")

In [3]:
output_dir.mkdir(exist_ok=True)  # Creates the directory to store output files

# View the schema

In [4]:
# Load the schema from the file
with open(json_schema_file, "r") as file:
    input_schema = json.load(file)

# Print the schema as pretty JSON
print(json.dumps(input_schema, indent=4))

{
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "BuildingType": {
            "type": "string",
            "enum": [
                "W1",
                "W2",
                "S1",
                "S2",
                "S3",
                "S4",
                "S5",
                "C1",
                "C2",
                "C3",
                "PC1",
                "PC2",
                "RM1",
                "RM2",
                "URM",
                "MH"
            ]
        },
        "DesignLevel": {
            "type": "string",
            "enum": [
                "Pre-Code",
                "Low-Code",
                "Moderate-Code",
                "High-Code"
            ]
        },
        "HeightClass": {
            "type": "string",
            "enum": [
                "Low-Rise",
                "Mid-Rise",
                "High-Rise"
            ]
        },
        "GroundFailure": {
       

# Process combinations and validate results

The schema provides a detailed list of attributes along with the possible values each attribute can assume. It also defines the rules for distinguishing valid and invalid combinations of these values, ensuring compatibility with the specific requirements of the fragility database for which the mapping function is designed. 

This step employs the `process_combinations` function from the `schema_mapping_validator` module to systematically generate all potential combinations of attribute values as defined by the schema. Each combination is validated according to the schema rules, and the mapping function is executed for combinations that meet the validation criteria.

The outcomes are classified into valid and invalid combinations, with the corresponding inputs and outputs saved in the output directory for further analysis.

In [5]:
start_time = time.time()
print("\nProcessing combinations and validating results...")
results = process_combinations(
    mapping_script, mapping_function, json_schema_file, parallel=True
)

valid_results_file = output_dir / "valid_combinations.json"
invalid_results_file = output_dir / "invalid_combinations.json"

with open(valid_results_file, "w") as valid_file:
    json.dump(results["valid"], valid_file, indent=4)

with open(invalid_results_file, "w") as invalid_file:
    json.dump(results["invalid"], invalid_file, indent=4)

print("Summary:")
print(f"Total combinations: {len(results['valid']) + len(results['invalid'])}")
print(f"Valid combinations: {len(results['valid'])}")
print(f"Invalid combinations: {len(results['invalid'])}")
print(f"Step completed in {time.time() - start_time:.2f} seconds")


Processing combinations and validating results...
Summary:
Total combinations: 66816
Valid combinations: 25088
Invalid combinations: 41728
Step completed in 78.46 seconds


# Count Model IDs from Valid Results

This step identifies unique model IDs generated by the mapping function, using the results from valid input combinations. The `count_model_ids` function from the `fragility_mapping_checker` module is used for this purpose. A summary of the model ID counts is then saved in the output directory for further analysis.

In [6]:
start_time = time.time()
print("\nCounting model IDs from valid results...")
model_id_counts_file = output_dir / "model_id_counts.json"
model_id_counts = count_model_ids(results["valid"], model_id_counts_file)

print(f"Total unique model IDs: {len(model_id_counts)}")
print(f"Step completed in {time.time() - start_time:.2f} seconds")


Counting model IDs from valid results...
Total unique model IDs: 132
Step completed in 0.01 seconds


# Check Fragility Mapping

This step verifies that all fragility model IDs in the provided database are successfully mapped by the valid combinations, using the `check_fragility_mapping` function from the `fragility_mapping_checker` module. The verification results are saved in the output directory, and a summary is displayed, emphasizing any unmapped IDs that require further attention.

In [7]:
start_time = time.time()
print("\nChecking fragility mapping...")
fragility_mapping_output_file = output_dir / "fragility_mapping_summary.json"
fragility_summary = check_fragility_mapping(
    results["valid"], fragility_csv, fragility_mapping_output_file
)

print("\nFragility Mapping Summary:")
print(f"Total # fragility model IDs: {fragility_summary['total_fragility_ids']}")
print(f"# fragility model IDs mapped into: {fragility_summary['mapped']}")
print(f"# fragility model IDs not mapped into: {fragility_summary['unmapped']}")
print("\nList of fragility model IDs not mapped into:")
for unmapped_id in fragility_summary["unmapped_ids"]:
    print(unmapped_id)
print(f"Step completed in {time.time() - start_time:.2f} seconds")


Checking fragility mapping...

Fragility Mapping Summary:
Total # fragility model IDs: 132
# fragility model IDs mapped into: 132
# fragility model IDs not mapped into: 0

List of fragility model IDs not mapped into:
Step completed in 0.01 seconds


In [8]:
# Display valid combinations without a fragility model ID
unmapped_model_ids = set(model_id_counts.keys()) - set(fragility_summary["mapped_ids"])
unmapped_model_ids_file = output_dir / "unmapped_model_ids.json"

with open(unmapped_model_ids_file, "w") as unmapped_file:
    json.dump(list(unmapped_model_ids), unmapped_file, indent=4)

print(f"\n{len(unmapped_model_ids)} valid results not mapped to a fragility model ID:")
for model_id in unmapped_model_ids:
    print(model_id)


0 valid results not mapped to a fragility model ID:
